In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib, os
matplotlib.use('Agg')
%matplotlib inline
from rfh_subsampling_analysis import *
import pandas as pd
import seaborn as sns
pd.set_option('precision', 4)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
            
model_directory = '../../../../rfh_importance_eval/'
class_dirs = [model_directory+'/{}/'.format(d) for d in os.listdir(model_directory) if os.path.isdir(model_directory+'/{}/'.format(d))]
k = 1

In [2]:
gather_df = gather_dir_metrics(model_directory, k, labels=['PriA-SSB AS'])
col_indices = list(range(6)) + list(range(9, 12)) + list(range(15, 63)) + list(range(87, 110))
col_indices = [i for i in col_indices if 'Mean' not in gather_df.columns[i] and 'Median' not in gather_df.columns[i]]
col_indices = [i for i in col_indices if 'PriA-SSB AS' in gather_df.columns[i]]
gather_df.columns = [col.replace('PriA-SSB AS', 'PriA-SSB prospective') for col in gather_df.columns]
metric_names = list(gather_df.columns.values[col_indices])
gather_df = gather_df[metric_names]
gather_df = gather_df.dropna(axis=0)
gather_df = gather_df.xs('test_metrics', level='set')
gather_df = gather_df.drop('Folds Mean', level='fold')
gather_df = gather_df.drop('Folds Median', level='fold')
gather_df = gather_df.reset_index(level=1, drop=True)

In [3]:
gather_df

,ROC AUC PriA-SSB prospective,BEDROC AUC PriA-SSB prospective,PR auc.integral PriA-SSB prospective,NEF_0.1 % PriA-SSB prospective,NEF_0.15 % PriA-SSB prospective,NEF_0.5 % PriA-SSB prospective,NEF_1 % PriA-SSB prospective,NEF_2 % PriA-SSB prospective,NEF_5 % PriA-SSB prospective,NEF_10 % PriA-SSB prospective,NEF_20 % PriA-SSB prospective,EF_0.1 % PriA-SSB prospective,EF_0.15 % PriA-SSB prospective,EF_0.5 % PriA-SSB prospective,EF_1 % PriA-SSB prospective,EF_2 % PriA-SSB prospective,EF_5 % PriA-SSB prospective,EF_10 % PriA-SSB prospective,EF_20 % PriA-SSB prospective,NEF AUC PriA-SSB prospective,n_hits_100 PriA-SSB prospective,n_hits_250 PriA-SSB prospective,n_hits_500 PriA-SSB prospective,n_hits_1000 PriA-SSB prospective,n_hits_2500 PriA-SSB prospective,n_hits_5000 PriA-SSB prospective,n_hits_10000 PriA-SSB prospective
model,,,,,,,,,,,,,,,,,,,,,,,,,,,
rule_0,0.8737,0.7261,0.4065,0.6818,0.5758,0.5556,0.6481,0.6667,0.7222,0.7778,0.7778,277.7778,234.5679,111.1111,64.8148,33.3333,14.4444,7.7778,3.8889,0.7399,29.0,35.0,37.0,38.0,42.0,43.0,48.0
rule_1,0.8835,0.7042,0.3293,0.5909,0.5758,0.4815,0.5370,0.6111,0.7037,0.7407,0.8148,240.7407,234.5679,96.2963,53.7037,30.5556,14.0741,7.4074,4.0741,0.7202,25.0,30.0,34.0,37.0,40.0,45.0,48.0
rule_2,0.8893,0.7290,0.3246,0.5909,0.6061,0.4630,0.5741,0.6481,0.7222,0.7778,0.8333,240.7407,246.9136,92.5926,57.4074,32.4074,14.4444,7.7778,4.1667,0.7474,24.0,31.0,36.0,39.0,45.0,45.0,47.0
rule_3,0.8741,0.7309,0.4485,0.7273,0.6364,0.5556,0.6296,0.6667,0.7222,0.7778,0.8333,296.2963,259.2593,111.1111,62.9630,33.3333,14.4444,7.7778,4.1667,0.7538,28.0,35.0,36.0,39.0,43.0,45.0,46.0
rule_4,0.8788,0.7341,0.4584,0.6818,0.6667,0.5556,0.6481,0.6667,0.7222,0.7963,0.8333,277.7778,271.6049,111.1111,64.8148,33.3333,14.4444,7.9630,4.1667,0.7617,29.0,35.0,36.0,39.0,43.0,45.0,48.0
rule_5,0.8852,0.7370,0.4731,0.7727,0.6667,0.5926,0.6481,0.6667,0.7222,0.7778,0.8333,314.8148,271.6049,118.5185,64.8148,33.3333,14.4444,7.7778,4.1667,0.7556,31.0,36.0,37.0,38.0,42.0,45.0,48.0
rule_6,0.8840,0.7410,0.4935,0.8182,0.6970,0.5926,0.6667,0.6852,0.7222,0.7963,0.8333,333.3333,283.9506,118.5185,66.6667,34.2593,14.4444,7.9630,4.1667,0.7655,32.0,36.0,37.0,39.0,43.0,45.0,48.0


---
# top250 Results

In [4]:
# load training data
file_list = ['../../../dataset/fixed_dataset/fold_5/file_{}.csv'.format(i) for i in range(5)]
train_pd = read_merged_data(file_list)
test_pd = pd.read_csv('../../../dataset/pria_prospective.csv.gz', compression='gzip')

y_train = train_pd['Keck_Pria_AS_Retest'].as_matrix()
y_test = test_pd['Keck_Pria_AS_Retest'].as_matrix()
# load training indices
train_indices_dict = {}
for model in gather_df.index:
    train_indices_file = model_directory+'/{}/fold_0/train_indices.npy'.format(model)
    train_indices_dict[model] = np.load(train_indices_file)

In [5]:
for model in gather_df.index:
    actives_count = np.where(y_train[train_indices_dict[model]] == 1)[0].shape[0]
    inactives_count = np.where(y_train[train_indices_dict[model]] == 0)[0].shape[0]
    top250_score = gather_df['n_hits_250 PriA-SSB prospective'][model]
    print('{}: actives count: {}\t inactives count: {}\t top250: {}'.format(model, actives_count, inactives_count, top250_score))

rule_0: actives count: 19	 inactives count: 1372	 top250: 35.0
rule_1: actives count: 23	 inactives count: 20	 top250: 30.0
rule_2: actives count: 29	 inactives count: 28	 top250: 31.0
rule_3: actives count: 20	 inactives count: 109	 top250: 35.0
rule_4: actives count: 20	 inactives count: 221	 top250: 35.0
rule_5: actives count: 20	 inactives count: 316	 top250: 36.0
rule_6: actives count: 20	 inactives count: 460	 top250: 36.0


In [6]:
overlapping_inactives = []
for rule_i in train_indices_dict:
    s_i = set(train_indices_dict[rule_i][np.where(y_train[train_indices_dict[rule_i]] == 0)[0]])
    for rule_j in train_indices_dict:
        s_j = set(train_indices_dict[rule_j][np.where(y_train[train_indices_dict[rule_j]] == 0)[0]])
        if rule_i != rule_j and s_i.issubset(s_j):
            overlapping_inactives.append((rule_i, rule_j))
overlapping_inactives

[('rule_2', 'rule_3'),
 ('rule_2', 'rule_6'),
 ('rule_2', 'rule_4'),
 ('rule_2', 'rule_5'),
 ('rule_1', 'rule_2'),
 ('rule_1', 'rule_3'),
 ('rule_1', 'rule_6'),
 ('rule_1', 'rule_4'),
 ('rule_1', 'rule_5'),
 ('rule_3', 'rule_6'),
 ('rule_3', 'rule_4'),
 ('rule_3', 'rule_5'),
 ('rule_4', 'rule_6'),
 ('rule_4', 'rule_5'),
 ('rule_5', 'rule_6')]

In [7]:
overlapping_actives = []
for rule_i in train_indices_dict:
    s_i = set(train_indices_dict[rule_i][np.where(y_train[train_indices_dict[rule_i]] == 1)[0]])
    for rule_j in train_indices_dict:
        s_j = set(train_indices_dict[rule_j][np.where(y_train[train_indices_dict[rule_j]] == 1)[0]])
        if rule_i != rule_j and s_i.issubset(s_j):
            overlapping_actives.append((rule_i, rule_j))
overlapping_actives

[('rule_1', 'rule_2'),
 ('rule_3', 'rule_2'),
 ('rule_3', 'rule_1'),
 ('rule_3', 'rule_6'),
 ('rule_3', 'rule_4'),
 ('rule_3', 'rule_5'),
 ('rule_6', 'rule_2'),
 ('rule_6', 'rule_1'),
 ('rule_6', 'rule_3'),
 ('rule_6', 'rule_4'),
 ('rule_6', 'rule_5'),
 ('rule_4', 'rule_2'),
 ('rule_4', 'rule_1'),
 ('rule_4', 'rule_3'),
 ('rule_4', 'rule_6'),
 ('rule_4', 'rule_5'),
 ('rule_5', 'rule_2'),
 ('rule_5', 'rule_1'),
 ('rule_5', 'rule_3'),
 ('rule_5', 'rule_6'),
 ('rule_5', 'rule_4')]

rules 1 to 6 actives/inactives are subsets as seen above. rule_0 contains some actives and inactives that appear in the other rules. 

rule_3 shows good performance with only 20 actives and 109 inactives. rule_5 goes up to 316	inactives in order to get 1 extra hit. rule_4 did not improve on rule_3, and so, we might be able to discard those inactives that are added to rule_4 and focus on the inactives that were added for rule_5.  

# Focus on rule_3 and rule_5 join

In [8]:
s3 = set(train_indices_dict['rule_3'][np.where(y_train[train_indices_dict['rule_3']] == 0)[0]])
s4 = set(train_indices_dict['rule_4'][np.where(y_train[train_indices_dict['rule_4']] == 0)[0]])
s5 = set(train_indices_dict['rule_5'][np.where(y_train[train_indices_dict['rule_5']] == 0)[0]])

s5_only = s5.difference(s4)
important_train_inactives = np.array(list(s5_only)+list(s3))
important_train_actives = train_indices_dict['rule_3'][np.where(y_train[train_indices_dict['rule_3']] == 1)[0]]
for i in range(important_train_inactives.shape[0]):
    important_train_inactives = important_train_inactives[:i+1]
    important_train_indices = np.concatenate([important_train_actives, important_train_inactives])
    np.save('./important_indices/rule_345_{}'.format(i), important_train_indices)